In [99]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

In [100]:
# Code Generation
message = "1101"  # 4-bit binary message

nr_codewords = int(100)
bits_info = torch.randint(2, (nr_codewords, 4), dtype=torch.int)

print(bits_info)

tensor([[1, 1, 0, 0],
        [0, 1, 0, 0],
        [1, 1, 1, 0],
        [1, 1, 1, 1],
        [1, 1, 0, 1],
        [1, 0, 1, 0],
        [1, 0, 1, 0],
        [1, 1, 0, 1],
        [0, 1, 1, 1],
        [1, 1, 1, 1],
        [0, 0, 0, 1],
        [1, 1, 1, 1],
        [0, 0, 1, 0],
        [1, 1, 0, 0],
        [0, 1, 1, 1],
        [0, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 1],
        [1, 1, 1, 0],
        [0, 1, 0, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1],
        [0, 1, 1, 0],
        [1, 0, 0, 1],
        [1, 1, 0, 0],
        [0, 1, 1, 1],
        [1, 0, 0, 1],
        [0, 0, 1, 1],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 0],
        [1, 1, 0, 1],
        [1, 1, 0, 1],
        [0, 0, 1, 1],
        [1, 1, 0, 0],
        [0, 1, 1, 1],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 1, 1],
        [1, 1, 0, 1],
        [0, 1, 0, 1],
        [1, 1, 1, 0],
        [1, 1, 1, 1],
        [0, 0, 1, 1],
        [1, 0, 1, 0],
        [0

Hamming(7,4) Encoder

In [101]:
class hamming_encode(torch.nn.Module):
    def __init__(self):
        """
            Use Hamming(7,4) to encode the data.
    
        Args:
            data: data received from the Hamming(7,4) encoder(Tensor)
            generator matrix: generate the parity code
    
        Returns:
            encoded data: 4 bits original info with 3 parity code.
        """
        super(hamming_encode, self).__init__()

        # Define the generator matrix for Hamming(7,4)
        self.generator_matrix = torch.tensor([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [1, 1, 0, 1],
            [1, 0, 1, 1],
            [0, 1, 1, 1],
        ], dtype=torch.int)

    def forward(self, input_data):
        # Ensure input_data has shape (batch_size, 4)
        assert input_data.size(1) == 4, "Input data must have 4 bits."

        # Perform matrix multiplication to encode the data
        encoded_data = torch.matmul(input_data, self.generator_matrix.t()) % 2

        return encoded_data

In [102]:
encoder = hamming_encode()
encoded_codeword = encoder(bits_info)
print(encoded_codeword)

tensor([[1, 1, 0, 0, 0, 1, 1],
        [0, 1, 0, 0, 1, 0, 1],
        [1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 0, 1, 1, 0, 0],
        [1, 0, 1, 0, 1, 0, 1],
        [1, 0, 1, 0, 1, 0, 1],
        [1, 1, 0, 1, 1, 0, 0],
        [0, 1, 1, 1, 0, 0, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 0, 0, 1, 1],
        [1, 1, 0, 0, 0, 1, 1],
        [0, 1, 1, 1, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 1, 1],
        [0, 0, 1, 1, 1, 0, 0],
        [1, 1, 1, 0, 0, 0, 0],
        [0, 1, 0, 1, 0, 1, 0],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [0, 1, 1, 0, 1, 1, 0],
        [1, 0, 0, 1, 0, 0, 1],
        [1, 1, 0, 0, 0, 1, 1],
        [0, 1, 1, 1, 0, 0, 1],
        [1, 0, 0, 1, 0, 0, 1],
        [0, 0, 1, 1, 1, 0, 0],
        [0, 1, 0, 0, 1, 0, 1],
        [0, 0, 1, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 1, 1, 0, 0],
        

BPSK Modulator + Noise

In [103]:
class bpsk_modulator(torch.nn.Module):
    def __init__(self):
        """
        Use BPSK to compress the data, which is easily to transmit.

    Args:
        codeword: data received from the Hamming(7,4) encoder(Tensor)

    Returns:
        data: Tensor contain all data modulated and add noise
    """
        super(bpsk_modulator, self).__init__()
        
    def forward(self, codeword, snr_dB):
        
        # data = torch.tensor(data, dtype=float)
        data = codeword.to(dtype=torch.float)
    
        for i in range(data.shape[0]):
            bits = data[i]
            bits = 2 * bits - 1
        
            # Add Gaussian noise to the signal
            noise_power = torch.tensor(10**(snr_dB / 10))
            noise = torch.sqrt(1/(2*noise_power)) * torch.randn(len(bits))
            noised_signal = bits + noise
            # noised_signal = bits
            data[i] = noised_signal
    
       
        return data

In [104]:
snr_dB = 15  # Signal-to-noise ratio in dB

# Modulate the signal
modulator = bpsk_modulator()
modulated_noise_signal = modulator(encoded_codeword, snr_dB)
print(modulated_noise_signal)

tensor([[ 1.1178,  0.7641, -1.1590, -0.9914, -1.1616,  1.2352,  0.7967],
        [-1.1743,  0.9289, -1.1130, -1.0890,  1.3767, -1.0006,  1.0304],
        [ 1.0611,  1.3639,  0.9586, -0.8155, -0.8542, -1.2515, -1.0057],
        [ 1.0507,  1.0226,  1.2421,  0.9811,  0.9083,  1.0349,  1.1440],
        [ 0.9858,  1.1053, -0.9180,  0.8954,  0.9971, -0.9813, -1.1855],
        [ 1.0569, -1.0731,  0.9862, -0.9973,  1.1435, -1.0183,  1.0840],
        [ 0.7746, -0.9547,  1.1740, -0.9498,  0.9732, -1.1921,  1.2294],
        [ 0.9248,  0.9976, -1.0757,  1.0388,  0.8792, -1.1182, -1.1299],
        [-1.0958,  1.1476,  0.7462,  0.8692, -0.9590, -0.9575,  1.1111],
        [ 0.9497,  0.9691,  0.9618,  0.9559,  0.7692,  0.7653,  0.9162],
        [-1.0312, -1.3513, -0.8714,  0.7506,  1.1461,  1.1211,  0.9311],
        [ 0.8523,  1.0548,  1.0699,  1.0493,  0.8810,  1.0152,  0.8994],
        [-1.1677, -0.9276,  1.0346, -1.1288, -1.1378,  1.0754,  1.0180],
        [ 0.9019,  0.8789, -1.0517, -0.9062, -0.756

LLR Log-likelihood
y = s + n
Assuming that \( s \) is equally likely to be 0 or 1, and \( n \) is Gaussian with zero mean and variance \( N_0/2 \), where \( N_0 \) is the noise power spectral density.


In [105]:
def llr(signal, snr):
    """
    Calculate Log Likelihood Ratio (LLR) for a simple binary symmetric channel.

    Args:
        signal (torch.Tensor): Received signal from BPSK.
        noise_std (float): Standard deviation of the noise.

    Returns:
        llr: Log Likelihood Ratio (LLR) values.
    """
    # Assuming Binary Phase Shift Keying (BPSK) modulation
    noise_std = torch.sqrt(torch.tensor(10**(snr / 10)))

    # Calculate the LLR
    llr = 2 * signal * noise_std

    # return llr_values, llr
    return llr


In [106]:
llr_output = llr(modulated_noise_signal, snr_dB)
print("LLR values:", llr_output)

LLR values: tensor([[ 12.5718,   8.5939, -13.0347, -11.1497, -13.0645,  13.8926,   8.9599],
        [-13.2069,  10.4470, -12.5175, -12.2478,  15.4834, -11.2539,  11.5882],
        [ 11.9345,  15.3395,  10.7810,  -9.1719,  -9.6070, -14.0749, -11.3110],
        [ 11.8167,  11.5013,  13.9698,  11.0342,  10.2156,  11.6389,  12.8660],
        [ 11.0873,  12.4306, -10.3242,  10.0703,  11.2142, -11.0367, -13.3332],
        [ 11.8873, -12.0688,  11.0912, -11.2168,  12.8611, -11.4526,  12.1912],
        [  8.7123, -10.7373,  13.2033, -10.6819,  10.9452, -13.4075,  13.8268],
        [ 10.4013,  11.2198, -12.0979,  11.6830,   9.8883, -12.5762, -12.7081],
        [-12.3244,  12.9064,   8.3920,   9.7759, -10.7860, -10.7688,  12.4966],
        [ 10.6813,  10.8997,  10.8173,  10.7506,   8.6506,   8.6074,  10.3041],
        [-11.5979, -15.1980,  -9.8010,   8.4417,  12.8903,  12.6084,  10.4717],
        [  9.5860,  11.8634,  12.0328,  11.8013,   9.9081,  11.4179,  10.1151],
        [-13.1334, -10.4321,

LDPC Decoder

In [131]:
class LDPCBeliefPropagation(torch.nn.Module):
    def __init__(self, H):
        """
        LDPC Belief Propagation.
    
        Args:
            H: Low density parity code for building tanner graph.
            llr: Log Likelihood Ratio (LLR) values. Only for 7-bit codeword.
    
        Returns:
            estimated_bits: the output result from belief propagation.
        """
        super(LDPCBeliefPropagation, self).__init__()
        self.H = H
        self.num_check_nodes, self.num_variable_nodes = H.shape

        # Initialize messages
        self.messages_v_to_c = torch.ones((self.num_variable_nodes, self.num_check_nodes),dtype=torch.float)
        self.messages_c_to_v = torch.zeros((self.num_check_nodes, self.num_variable_nodes),dtype=torch.float)

    def forward(self, llr, max_iter):
        for iteration in range(max_iter):
            # Variable to check node messages
            for i in range(self.num_variable_nodes):
                for j in range(self.num_check_nodes):
                    # Compute messages from variable to check nodes
                    connected_checks = self.H[j, :] == 1
                    product = torch.prod(torch.tanh(0.5 * self.messages_v_to_c[connected_checks, j]))
                    self.messages_v_to_c[i, j] = torch.sign(llr[i]) * product

            # Check to variable node messages
            for i in range(self.num_check_nodes):
                for j in range(self.num_variable_nodes):
                    # Compute messages from check to variable nodes
                    connected_vars = self.H[:, j] == 1
                    sum_msgs = torch.sum(self.messages_c_to_v[connected_vars, i]) - self.messages_v_to_c[j, i]
                    self.messages_c_to_v[i, j] = 2 * torch.atan(torch.exp(0.5 * sum_msgs))

        # Calculate the final estimated bits and only take first four bits
        estimated_bits = torch.sign(llr) * torch.prod(torch.tanh(0.5 * self.messages_c_to_v), dim=0)
        estimated_bits = torch.where(estimated_bits>0, torch.tensor(1), torch.tensor(0))
        estimated_bits = estimated_bits[0:4]


        return estimated_bits
    

In [132]:
# Define LDPC parameters
H = torch.tensor([ [1, 1, 1, 0, 0, 0, 0],
                   [0, 0, 1, 1, 1, 0, 0],
                   [0, 1, 0, 0, 1, 1, 0],
                   [1, 0, 0, 1, 0, 0, 1],])
iter = 50
ldpc_bp = LDPCBeliefPropagation(H)

# Store the final result from LDPC
tensor_size = torch.Size([llr_output.shape[0], 4])
final_result = torch.zeros(tensor_size)

#loop all the llr and get result.
for i in range(llr_output.shape[0]):
    bp_data = llr_output[i]
    estimated_bits = ldpc_bp(bp_data, iter)
    final_result[i] = estimated_bits

print(final_result)

# llr_demodulator_output = llr_output
# 
# estimated_bits = ldpc_bp(llr_demodulator_output, iter)
# 
# print("LLR Demodulator Output:", llr_demodulator_output)
# print("Estimated Bits:", estimated_bits)

tensor([[1., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.],
        [1., 1., 0., 1.],
        [1., 0., 1., 0.],
        [1., 0., 1., 0.],
        [1., 1., 0., 1.],
        [0., 1., 1., 1.],
        [1., 1., 1., 1.],
        [0., 0., 0., 1.],
        [1., 1., 1., 1.],
        [0., 0., 1., 0.],
        [1., 1., 0., 0.],
        [0., 1., 1., 1.],
        [0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 1.],
        [1., 1., 1., 0.],
        [0., 1., 0., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [0., 1., 1., 0.],
        [1., 0., 0., 1.],
        [1., 1., 0., 0.],
        [0., 1., 1., 1.],
        [1., 0., 0., 1.],
        [0., 0., 1., 1.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [1., 1., 0., 1.],
        [1., 1., 0., 1.],
        [0., 0., 1., 1.],
        [1., 1., 0., 0.],
        [0., 1., 1., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0.,

Comparation and Plot